# Generate Tim Fletcher Style Therapeutic Conversations

**GPU**: Google Colab Free Tier (T4)  
**Purpose**: Generate synthetic therapeutic conversations in Tim Fletcher's teaching voice  
**Input**: Tim Fletcher voice profile (extracted from 913 transcripts)  
**Output**: Training dataset for Stage 3 (voice injection)

---

## Setup

**Before running**:
1. Runtime → Change runtime type → T4 GPU
2. Upload `tim_fletcher_voice_profile.json` to Colab
3. Set your HuggingFace token (for model access)

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
%pip install -q transformers accelerate bitsandbytes scipy
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from typing import List, Dict
import random
from tqdm.auto import tqdm

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

## Load Tim Fletcher Voice Profile

In [ ]:
# Upload tim_fletcher_voice_profile.json first, or load from Google Drive
# Option 1: Upload file
from google.colab import files
uploaded = files.upload()

# Load voice profile
with open('tim_fletcher_voice_profile.json', 'r') as f:
    voice_profile = json.load(f)

print("Voice Profile Loaded:")
print(f"  Sentence starters: {len(voice_profile['sentence_starters'])}")
print(f"  Transition phrases: {len(voice_profile['transition_phrases'])}")
print(f"  Empathy markers: {len(voice_profile['empathy_markers'])}")
print(f"  Analogies: {len(voice_profile['analogies'])}")
print(f"  Examples: {len(voice_profile['examples'])}")

## Load Language Model (4-bit Quantized for Free Tier)

In [ ]:
# Model selection - using a good instruction-following model
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"  # Fast and good quality
# Alternative: "meta-llama/Llama-2-7b-chat-hf" (requires HF token)

# 4-bit quantization config for free GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Model loaded successfully!")
print(f"Model memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")

## Create Tim Fletcher Voice System Prompt

In [ ]:
def create_tim_fletcher_prompt(voice_profile: Dict) -> str:
    """Create system prompt that captures Tim's voice"""

    prompt = """You are a therapeutic AI assistant trained to speak in Tim Fletcher's teaching voice.

TIM FLETCHER'S VOICE CHARACTERISTICS:

SENTENCE STARTERS (use frequently):
"""
    # Top 10 sentence starters
    starters = list(voice_profile['sentence_starters'].items())[:10]
    for starter, count in starters:
        prompt += f"- \"{starter}...\"\n"

    prompt += "\nTRANSITION PHRASES (connect ideas):\n"
    for phrase in list(voice_profile['transition_phrases'].keys())[:8]:
        prompt += f"- \"{phrase}\"\n"

    prompt += "\nEMPATHY MARKERS (show understanding):\n"
    for marker in list(voice_profile['empathy_markers'].keys())[:8]:
        prompt += f"- \"{marker}\"\n"

    prompt += """\nTEACHING STYLE:
- Break down complex concepts into simple steps
- Use numbered points (First, Second, Third)
- Give concrete examples with \"Let's say\" or \"Think about\"
- Use analogies: \"It's like...\" or \"Imagine...\"
- Normalize experiences: \"Many people...\" \"For some people...\"
- Show deep empathy and understanding
- Connect to real-life scenarios
- Focus on complex trauma, PTSD, recovery concepts

EXAMPLE ANALOGIES:
"""
    for analogy in voice_profile['analogies'][:3]:
        prompt += f"- {analogy}\n"

    prompt += "\nEXAMPLE EXPRESSIONS:\n"
    for example in voice_profile['examples'][:3]:
        prompt += f"- {example}\n"

    prompt += """\nYour responses should:
1. Sound natural and conversational (like Tim speaking)
2. Show deep understanding of complex trauma
3. Use Tim's characteristic phrases and transitions
4. Include examples and analogies
5. Be compassionate and educational
"""

    return prompt

# Create the prompt
tim_system_prompt = create_tim_fletcher_prompt(voice_profile)
print("✅ Tim Fletcher voice prompt created")
print(f"\nPrompt preview (first 500 chars):\n{tim_system_prompt[:500]}...")

## Conversation Topics (Complex Trauma Focus)

In [ ]:
# Topics from Tim Fletcher's teachings
CONVERSATION_TOPICS = [
    # Complex Trauma Characteristics
    "fear of abandonment and rejection",
    "difficulty trusting others",
    "emotional dysregulation and mood swings",
    "shame and negative self-identity",
    "hypervigilance and always being on alert",
    "difficulty with change and transitions",
    "people-pleasing and loss of self",
    "impulsive behaviors and self-sabotage",
    "dissociation and feeling numb",
    "maladaptive coping mechanisms",

    # PTSD & Recovery
    "flashbacks and intrusive memories",
    "triggers and how to manage them",
    "building healthy attachments",
    "learning to regulate emotions",
    "processing childhood wounds",
    "healing from narcissistic abuse",
    "breaking trauma bonds",
    "developing self-compassion",
    "understanding the limbic brain vs cortex",
    "creating safety and stability",

    # Relationships
    "fear of intimacy and vulnerability",
    "setting healthy boundaries",
    "recognizing unhealthy relationship patterns",
    "dealing with conflict in relationships",
    "impact of trauma on parenting",
    "recovering from codependency",

    # Specific Issues
    "anger and resentment from past hurt",
    "grief and loss in trauma recovery",
    "perfectionism as a trauma response",
    "addiction and compulsive behaviors",
    "anxiety and panic attacks",
    "depression and hopelessness",
    "spiritual crisis after trauma",
    "forgiving those who hurt you",
    "rebuilding identity after trauma",
]

print(f"📋 {len(CONVERSATION_TOPICS)} conversation topics loaded")

## Client Message Templates

In [ ]:
# Realistic client opening messages
CLIENT_TEMPLATES = [
    "I've been struggling with {topic}. It's been affecting my daily life.",
    "Can you help me understand {topic}? I don't know where to start.",
    "I think I'm dealing with {topic}, but I'm not sure what to do about it.",
    "I've noticed patterns of {topic} in my life. How do I break free from this?",
    "I'm having a really hard time with {topic}. It feels overwhelming.",
    "I grew up in a difficult environment and now I'm experiencing {topic}.",
    "Ever since I can remember, I've dealt with {topic}. Is this related to my childhood?",
    "I want to heal from {topic}, but I don't know how.",
    "My therapist mentioned {topic}. Can you explain what that means?",
    "I keep finding myself stuck in {topic}. What's happening to me?",
]

# Follow-up client responses
CLIENT_FOLLOWUPS = [
    "That makes sense. Can you give me an example?",
    "I think I understand. But what do I do when this gets triggered?",
    "This is really helpful. How long does healing from this usually take?",
    "I've tried to work on this before but always fall back. Why does that happen?",
    "What you're saying resonates with me. Where should I start?",
    "I'm worried I'll never get better. Is that normal to feel?",
    "This is hard to hear, but it's true. What's the next step?",
    "I can see how my childhood relates to this now. How do I move forward?",
]

print(f"✅ Client templates loaded")

## Conversation Generator

In [ ]:
def generate_therapist_response(client_message: str, conversation_history: List[Dict], max_length: int = 512) -> str:
    """Generate therapist response in Tim Fletcher's voice"""

    # Build conversation context
    messages = [
        {"role": "system", "content": tim_system_prompt},
    ]

    # Add conversation history
    for turn in conversation_history:
        role = "user" if turn["role"] == "client" else "assistant"
        messages.append({"role": role, "content": turn["content"]})

    # Add current client message
    messages.append({"role": "user", "content": client_message})

    # Format for model
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response.strip()


def generate_conversation(topic: str, num_turns: int = 4) -> Dict:
    """Generate a full therapeutic conversation"""

    conversation = []

    # Client opening
    opening_template = random.choice(CLIENT_TEMPLATES)
    client_opening = opening_template.format(topic=topic)
    conversation.append({"role": "client", "content": client_opening})

    # Generate multi-turn conversation
    for turn in range(num_turns):
        # Therapist response
        therapist_response = generate_therapist_response(
            conversation[-1]["content"],
            conversation[:-1]
        )
        conversation.append({"role": "therapist", "content": therapist_response})

        # Client follow-up (except on last turn)
        if turn < num_turns - 1:
            client_followup = random.choice(CLIENT_FOLLOWUPS)
            conversation.append({"role": "client", "content": client_followup})

    return {
        "conversation": conversation,
        "metadata": {
            "source": "tim_fletcher_synthetic",
            "topic": topic,
            "model": MODEL_NAME,
            "turns": len(conversation)
        }
    }

print("✅ Conversation generator ready")

## Test Generation (Single Conversation)

In [ ]:
# Test with one conversation
print("🧪 Testing conversation generation...\n")

test_topic = random.choice(CONVERSATION_TOPICS)
print(f"Topic: {test_topic}\n")

test_conversation = generate_conversation(test_topic, num_turns=2)

# Display conversation
for i, turn in enumerate(test_conversation['conversation'], 1):
    role = turn['role'].upper()
    content = turn['content']
    print(f"[{role}]: {content}\n")

print("\n✅ Test successful!")

## Generate Full Dataset (1000 Conversations)

In [ ]:
# Configuration
NUM_CONVERSATIONS = 1000
TURNS_PER_CONVERSATION = 4  # 4-8 exchanges

print(f"🎨 Generating {NUM_CONVERSATIONS} conversations...")
print(f"   Turns per conversation: {TURNS_PER_CONVERSATION}")
print(f"   Topics: {len(CONVERSATION_TOPICS)}")
print("\nThis will take approximately 2-3 hours on free T4 GPU\n")

conversations = []
errors = []

for i in tqdm(range(NUM_CONVERSATIONS), desc="Generating conversations"):
    try:
        # Random topic
        topic = random.choice(CONVERSATION_TOPICS)

        # Random number of turns (4-6)
        num_turns = random.randint(4, 6)

        # Generate conversation
        conversation = generate_conversation(topic, num_turns=num_turns)
        conversations.append(conversation)

        # Save checkpoint every 100 conversations
        if (i + 1) % 100 == 0:
            checkpoint_file = f'tim_fletcher_conversations_checkpoint_{i+1}.json'
            with open(checkpoint_file, 'w', encoding='utf-8') as f:
                json.dump(conversations, f, indent=2, ensure_ascii=False)
            print(f"\n💾 Checkpoint saved: {checkpoint_file}")

    except Exception as e:
        print(f"\n⚠️ Error on conversation {i}: {e}")
        errors.append({"index": i, "error": str(e)})
        continue

print(f"\n✅ Generation complete!")
print(f"   Successfully generated: {len(conversations)} conversations")
print(f"   Errors: {len(errors)}")

## Save Final Dataset

In [ ]:
# Save final dataset
output_file = 'tim_fletcher_synthetic_conversations.json'

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(conversations, f, indent=2, ensure_ascii=False)

print(f"💾 Saved {len(conversations)} conversations to {output_file}")

# Calculate statistics
total_turns = sum(len(c['conversation']) for c in conversations)
avg_turns = total_turns / len(conversations)

print(f"\n📊 Dataset Statistics:")
print(f"   Total conversations: {len(conversations)}")
print(f"   Total turns: {total_turns}")
print(f"   Average turns per conversation: {avg_turns:.1f}")
print(f"   Unique topics: {len(set(c['metadata']['topic'] for c in conversations))}")

# File size
import os
file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
print(f"   File size: {file_size_mb:.2f} MB")

## Download Dataset

In [ ]:
# Download to local machine
from google.colab import files

print("📥 Downloading dataset...")
files.download(output_file)
print("✅ Download complete!")
print("\nUpload this file to: /home/vivi/pixelated/ai/data/tim_fletcher_voice/synthetic_conversations.json")

## Sample Output Review

In [ ]:
# Display a few sample conversations
print("📋 Sample Conversations:\n")
print("=" * 80)

for i in range(min(3, len(conversations))):
    conv = conversations[i]
    print(f"\n### CONVERSATION {i+1}")
    print(f"Topic: {conv['metadata']['topic']}")
    print(f"Turns: {conv['metadata']['turns']}\n")

    for turn in conv['conversation']:
        role = turn['role'].upper()
        content = turn['content'][:200] + '...' if len(turn['content']) > 200 else turn['content']
        print(f"[{role}]: {content}\n")

    print("=" * 80)

---

## Next Steps

1. **Download** the generated dataset
2. **Upload** to `/home/vivi/pixelated/ai/data/tim_fletcher_voice/synthetic_conversations.json`
3. **Review** sample conversations for quality
4. **Update** Stage 3 training config to point to this dataset
5. **Run** Stage 3 training on Lightning.ai H100

**Expected Output**: 1000 therapeutic conversations in Tim Fletcher's teaching voice, ready for Stage 3 training